In [12]:
import random
import projetannuel

from lib_gridfs import load_posters_with_size, stock_metrics
imgs, genres = load_posters_with_size((15, 10), genres=['Horreur', 'Animation', 'Histoire'])
imgs_as_lists = [img.tolist() for img in imgs]
import numpy as np

imgs_as_lists = np.array(imgs_as_lists)/255.0

data = list(zip(imgs_as_lists, genres))
# filtered_data = [(img, genre) for img, genre in data if genre in ['Animation', 'Horreur']]
random.shuffle(data)
imgs_shuffled = [movie[0] for movie in data]
genres_shuffled = [movie[1] for movie in data]
# genres_shuffled_binarygenres_shuffled_binary = []
# for g in genres_shuffled:
#     .append(0 if g == "Horreur" else 1)
lim = int(len(data) * 0.8)
imgs_as_lists_train = imgs_shuffled[:lim]
genres_train = genres_shuffled[:lim]
imgs_as_lists_test = imgs_shuffled[lim:]
genres_test = genres_shuffled[lim:]
from scipy.spatial.distance import pdist
import numpy as np

def median_pairwise_squared_distance(X):
    dists = pdist(X, metric='sqeuclidean')
    return np.median(dists)

import numpy as np
from sklearn.metrics import pairwise_distances

def heuristic_c_from_interclass(X, y):
    classes = np.unique(y)
    centers = [X[y == c].mean(axis=0) for c in classes]
    dists = pairwise_distances(centers)
    median_dist = np.median(dists[np.triu_indices_from(dists, k=1)])
    return 1.0 / (median_dist**2 + 1e-6)  # Add epsilon to avoid div-by-zero

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
genres_encoded = le.fit_transform(genres_shuffled)

c_est = heuristic_c_from_interclass(np.array(imgs_shuffled), genres_encoded)

gamma = 1.0 / (2 * median_pairwise_squared_distance(imgs_shuffled))
gamma

print(c_est)

📦 Found 2962 images of size 15x10


📥 Loading resized images: 100%|██████████| 2962/2962 [00:02<00:00, 1051.94it/s]


0.13758400307472035


In [10]:
np.array(imgs_shuffled).shape

(2962, 450)

In [4]:
from projetannuel import SVM, SVMKernelType, SoftMargin

model = SVM(SVMKernelType.Linear(),SoftMargin.Hard())
model.train(imgs_as_lists_train,genres_train)
Y_pred = [model.predict([x])[0] for x in imgs_as_lists_test]
print(f"Accuracy Score : {projetannuel.accuracy_score(genres_test, Y_pred)}")

TypeError: argument 'y_py': must be real number, not str

In [4]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from projetannuel import SVMOvR, SVMKernelType, SoftMargin, SVMOvO

# 1. Generate synthetic multiclass data (3 classes, 150 samples, 10 features)
X, y = make_classification(
    n_samples=3000,
    n_features=450,
    n_informative=6,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

# 2. Encode y to ensure integer labels 0, 1, 2 (not strictly needed here, but for consistency)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 3. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# 4. Convert to Python lists
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
y_train_list = y_train.tolist()
y_test_list = y_test.tolist()

# 5. Choose kernel and margin
gamma = 1 / X.shape[1]  # ~1 / n_features
kernel = SVMKernelType.rbf(gamma)
margin = SoftMargin.soft(1.0)

# 6. Initialize and train model
model = SVMOvR(kernel, margin)
model.train(X_train_list, y_train_list)

# 7. Predict
y_pred = model.predict(X_test_list)

# 8. Evaluate
acc = accuracy_score(y_test_list, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")
print(f"🧪 True labels: {y_test_list}")
print(f"📈 Predicted:  {y_pred}")


OSQP status   = Solved(Solution { x: [-5.058517485443808e-22, 4.727042565167372e-21, 0.9508864112154891, 1.0, 0.9551566349603081, 1.0, 0.04151203841877072, 0.9247111793996784, 0.34012706723758546, 1.0, 0.9732122584957578, 0.9096521939241743, 1.0, 0.8593680895824716, 7.256514597930395e-21, 5.860486205631875e-21, -2.3043669498418293e-21, -3.707300971921856e-21, 0.8143106109088822, 1.0, -7.521323050037498e-22, 1.0, 4.829315320286904e-21, 0.2666685897939493, 5.896741073923803e-21, 0.24169329165762618, 0.13626129112786833, 0.21354877485322926, 0.45225239603650635, 0.13664770864123466, 0.15627748695186755, 0.6428008760429773, 1.0, 0.4432847152828701, 6.454252015667423e-21, -4.072799024423932e-21, 1.8363083476935903e-21, 0.3792103517247366, 2.2596983484811237e-21, 0.3191741643209815, 0.30971613088660016, 0.5110107230386582, 0.8836058678998411, 0.6724637920981976, -6.982596284399684e-21, 1.3699132703553043e-21, 1.0, 1.0804238049882228e-21, 0.9699655711077074, 0.5649685442192013, 0.413977454569

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from projetannuel import SVMOvO, SVMKernelType, SoftMargin, SVMOvO

# 1. Generate synthetic multiclass data (3 classes, 150 samples, 10 features)
X, y = make_classification(
    n_samples=3000,
    n_features=450,
    n_informative=6,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

# 2. Encode y to ensure integer labels 0, 1, 2 (not strictly needed here, but for consistency)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# 3. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# 4. Convert to Python lists
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
y_train_list = y_train.tolist()
y_test_list = y_test.tolist()

# 5. Choose kernel and margin
gamma = 1 / X.shape[1]  # ~1 / n_features
kernel = SVMKernelType.rbf(gamma)
margin = SoftMargin.soft(1.0)

# 6. Initialize and train model
model = SVMOvO(kernel, margin)
model.train(X_train_list, y_train_list)

# 7. Predict
y_pred = model.predict(X_test_list)

# 8. Evaluate
acc = accuracy_score(y_test_list, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")
print(f"🧪 True labels: {y_test_list}")
print(f"📈 Predicted:  {y_pred}")


OSQP status   = Solved(Solution { x: [0.24831932517497973, 0.8680588557268828, 0.9587549702324193, 0.5717877871352522, 1.0, 0.20174656376096065, 0.6953901166320741, 1.0, 1.0, 0.7001128976296334, 1.0, -1.10080966001667e-21, 0.5520510837860496, 0.34698639591006536, 1.0, 0.8903870977908085, 0.4563049826823255, 0.739290707357937, -8.520140332963933e-22, -1.1511333110848904e-21, -1.1041541654627765e-21, 0.06203637700814381, -1.3405886237453154e-21, -2.148690112231568e-21, -2.003063588855035e-21, 0.2080351252086726, 0.8884655770492196, 0.8522034549226702, 1.9019572823347894e-21, 0.6799660752209419, 0.019819530242649954, 0.4160673539891384, 0.9045148012345307, 3.064774782149941e-22, 0.5549089314675427, 0.30071191119359697, -3.795378891852346e-21, -1.0047797355682399e-21, 1.7492895604102375e-21, 1.0, 0.7281641523565945, 1.0, 2.935542528833383e-21, -4.091173672193568e-21, 2.2791313674730238e-21, 3.7382064050220213e-22, 0.7405196904983484, 1.0, 0.23127570759461094, 5.654233371404758e-22, 1.27256